# Multi-Objective Routing Path Optimization

## Abstract

## Introduction

| Distance d | Transmission rate (Mbps) | 
| ------ | ------ |
| d >= 3000m | 0 |
| 3000m > d >= 2500m | 1 |
| 2500m > d >= 2000m | 2 |
| 2000m > d >= 1500m | 3 |
| 1500m > d >= 1000m | 4 |
| 1000m > d >= 500m | 5 |
| 500m > d | 7 |

## Methodology

### Objective Function

### Results Visualization on a Map

To visualize the solutions produced by the various algorithms, we implemented a real and interactive geographic map with the sensors and the base station. This was done using Folium, a Leaflet JavaScript library which we interacted with through an API. We followed through the API documentation (Story 2013) to achieve the following:
1. Show the best route obtained from a sensor to a base station by a particular algorithm
2. Show all other possible routes found (not the best but have end-to-end connectivity)

The locations of the sensors were provided in form of X, Y grid coordinates in meters for purposed of calculating their heuristic distances. However, since the API needs values of geographic coordinates, coordinates of the sensors had to be computed and represented on the leaflet. The computation of geographic coordinates from X, Y grid coordinates was inspired by the Haversine formula (Scripts 2024).

`latitude = central latitude + y/110574`<br>
`longitude = central longitude + x/(11320 * cos(central latitude))`

where x and y are the x, y values of the sensors provided in the csv file while central latitude and central longitude are the (0, 0) coordinates on the grid from which all other sensor distances are calculated from. From the map provided in the graphic in the assignment instructions we estimated the geographic locations of the base stations in Lyndhurst and Beaulieu as shown in figure 1 by referring to google maps.

![Alt](https://github.com/samariwa/search-and-optimization-projects/blob/main/assessment/images/search%20area.png?raw=true)

The values ‘110574’ and ‘11320’ in the equations above are the distances between the latitudes and longitudes in meters respectively (Rosenberg 2024). With the geographic coordinates derived from the grid coordinates, we were able to feed the API with the sensor and base station information which was useful in plotting paths.

![Alt](https://github.com/samariwa/search-and-optimization-projects/blob/main/assessment/images/blank%20map.png?raw=true)

The visualization code functions are found in the [data and plotting module](./data_plot.ipynb) which is has all data fetching and manipulation functions as well as the map visualization functions.

#### Map Visualization Functionalities

<b>Data Fetching and constant variable initialization:</b> The X, Y grid data is fetched from the CSV file and all constants related to the data such as the ‘distance bandwidth data’ defined in the instructions are initialized. It is at this point that a blank map in the search area of interest is initialized.<br>
<b>Grid to Geographic coordinates conversion:</b> Using the formulas defined above, the X, Y data is converted to geographic coordinates using the function `get_geographic_coordinates()`<br>
<b>Initialization of a map with sensors:</b> Using the computed geographic coordinates, the sensors and base stations are represented on the blank map. This initialization happens every time an algorithm is executed for visualization to be done on a fresh map. This is done using the `init_blank_map()` function.<br>
<b>Plotting routes:</b> Joins the sensors that are part of a transmission path using lines from the origin node to the destination node. This is done for the best path found which is represented using a yellow line and all the other possible paths that are represented using a variety of colors. The ‘best path’ and ‘possible paths’ have been categorically layered in a way that you can view each type independently without struggle. For further visual intuition, the best path’s sensors have been marked with pins which show their sensor IDs. This can be done using a layer controller found on the top left corner. This functionality has been implemented in the `plot_route()` function. With takes in the coordinates as arguments with a specification of whether they are the best routes or they are possible routes.


In [17]:
![Alt](https://github.com/samariwa/search-and-optimization-projects/blob/main/assessment/images/map%20visualization.png?raw=true)

zsh:1: unknown file attribute: h


## Results

### Imports

In [20]:
import ast
import json
import scipy.stats as stat
%run data_plot.ipynb
%run metrics.ipynb
%run discrete_genetic_algorithm.ipynb
%run ant_colony_optimization.ipynb
%run dijkstras_algorithm.ipynb
%run simulated_annealing.ipynb

In [21]:
def parse_map_data(results):
    """
    parse_map_data(results)
    converts routes data to geographic routes that can be represented on a map.
    this is done by checking the sequence of x y coordinates against their corresponding
    geographic coordinates defined in the plotting module.
    results: the algorithm search results that contains the best solution to the destination
    and all other possible routes.
    returns: the coordinates used to create the map representation
    """
    # init dictionary with the structure expected for map visualization
    coordinates = {'best_route': {'sensor_id': [],
                              'coordinates': []}, 
               'possible_routes': []
              }
    # convert the string containing the list of the best path to type list
    possible_paths = [ast.literal_eval(paths) for paths in results['possible_paths'].keys()]
    # get the geographic coordinates for all nodes in possible paths
    for path in possible_paths:
        route_coords = []
        possible_path_dest_idx = len(path) - 1
        for idx, sensor in enumerate(path):
            if idx != 0 and idx != possible_path_dest_idx:
                # duplicate coordinates for all nodes but for the first and last
                route_coords.append(geographic_coords[str(sensor)])
                route_coords.append(geographic_coords[str(sensor)])
            else:
                route_coords.append(geographic_coords[str(sensor)])
        coordinates['possible_routes'].append(route_coords)
    # add best path last so that an visual it appears on top
    best_path = ast.literal_eval(list(results['best_path'].keys())[0])
    best_path_dest_idx = len(best_path) - 1
    for idx, sensor in enumerate(best_path):
        # include sensor IDs for the best path except for the destination (won't need pin marker)
        if idx != best_path_dest_idx:
            coordinates['best_route']['sensor_id'].append(sensor)
        if idx != 0 and idx != best_path_dest_idx:
            # duplicate coordinates for all nodes but for the first and last
            coordinates['best_route']['coordinates'].append(geographic_coords[str(sensor)])
            coordinates['best_route']['coordinates'].append(geographic_coords[str(sensor)])
        else:
            coordinates['best_route']['coordinates'].append(geographic_coords[str(sensor)])

    return coordinates

In [22]:
def load_solutions():
    """
    load_solutions()
    it loads solutions that already exist since new solutions are appended to the json
    object before being added written on the file.
    returns: the json object read from the file
    """
    # fetch existing best solutions from the solutions file
    with open('solutions.json', 'r') as file:
        try:
            solutions = json.load(file)
        except Exception as e:
            # if the solutions file is empty, initialize and empty dict to be used
            solutions = {}

    return solutions

In [23]:
def save_solution(solution, method):
    """
    save_solution(solution, method)
    saves the best solution found by each algorithm in a solutions json file.
    the solutions file has a format like...
    {"source node": "Node-5", 
    "routing path": "(Node-2, 1 Mbps), (Node-1, 4Mbps), (BS-2, 2 Mbps)",
    "end-to-end transmission rate": "1 Mbps"}
    solution: the best solution to a base station from a sensor
    method: the method that was used to get that best solution.
    returns: nothing
    """
    # fetch existing solutions
    existing_solutions = load_solutions()
    '''
    if solution for a sensor using the method specified exists, 
    update the dict with a new solution, otherwise create the solution
    '''
    if method in existing_solutions.keys():
        existing_solutions[method][solution[0]['source node']] = solution[0]
    else:
        existing_solutions[method] = {}
        existing_solutions[method][solution[0]['source node']] = solution[0]
    # write the updated solutions to the solutions file
    json.dump(existing_solutions, open( f"solutions.json", 'w' ) )

    return

In [24]:
def find_optimal_route(origin, method, **kwargs):
    """
    find_optimal_route(origin, method, **kwargs)
    the main function of this assessment. It searches for best route 
    from a node to either of the 2 base stations. The search is done using
    the algorithm specified as one of the arguments. Supported algorithms 
    include: 'discrete genetic algorithm', 'ant colony optimization', 
    'dijkstras algorithm' and 'simulated annealing'. Each of the algorithms,
    return a best solutions dictionary that is stored in the solutions file,
    and the results dictionary which is used for visualization on a map
    origin: the node whose best path to a base station you are seeking
    method: the algorithm you want to use to perform the search
    **kwargs: any additional arguments you want to pass in for
    optimization of the various algorithms
    Expected **kwargs and their fallback values:
    discrete genetic algorithm: pop_length=100000, generations=10, mutation_rate=0.1, crossover_type='single-point', selection_limit=20
    ant colony optimization: Q=1, alpha = 0.6, evaporate=0.45, ants_count=20, max_iterations=10
    dijkstras algorithm: None
    simulated annealing: init_temp=100, final_temp=0.1, cooling_rate=0.95, improvement_checker_count=50
    More details on the **kwargs provided in the algorithms' respective modules
    returns: nothing
    """
    # run the algorithm passed in as a parameter and fetch the visualization results together with the solutions file data
    if method == "discrete genetic algorithm":
        results, solutions = discrete_genetic_algorithm(
            origin,
            pop_length=kwargs.get("pop_length", 100000),
            generations=kwargs.get("generations", 10),
            mutation_rate=kwargs.get("mutation_rate", 0.1),
            crossover_type=kwargs.get("crossover_type", "single-point"),
            selection_limit=kwargs.get("selection_limit", 20),
        )
    elif method == "dijkstras algorithm":
        results, solutions = ant_colony_optimization(
            origin,
            Q=kwargs.get("Q", 1),
            alpha=kwargs.get("alpha", 0.6),
            evaporate=kwargs.get("evaporate", 0.45),
            ants_count=kwargs.get("ants_count", 20),
            max_iterations=kwargs.get("max_iterations", 10),
        )
    elif method == "ant colony optimization":
        results, solutions = ant_colony_optimization(origin)
    elif method == "simulated annealing":
        results, solutions = simulated_annealing(
            origin,
            init_temp=kwargs.get("init_temp", 100),
            final_temp=kwargs.get("final_temp", 0.1),
            cooling_rate=kwargs.get("cooling_rate", 0.95),
            improvement_checker_count=kwargs.get("improvement_checker_count", 50),
        )
    else:
        return "unknown algorithm"
    # save the best path in the solutions file
    save_solution(solutions, method)
    # parse the plotting results and call the map visualization function
    coordinates = parse_map_data(results)
    plot_route(coordinates)

    return

In [ ]:
find_optimal_route(43, 'discrete genetic algorithm', pop_length=100000)
#find_optimal_route(1, 'ant colony optimization')
#find_optimal_route(43, 'simulated annealing')

**********************************Generation 1**********************************


In [ ]:
def get_missing_solutions():
    """
    get_missing_solutions()
    for each algorithm, it gets the nodes that dont have any documented
    solutions to any of the based stations. This is based on the records
    found in the solutions json file.
    returns: a dictionary containing the missing solution for each algorithm
    """
    missing_solutions = {}
    sensors = list(x_y_data.keys())[:-2]
    solutions = load_solutions()

    for algorithm in solutions.keys():
        for sensor in sensors:
            if solutions[algorithm].get(f"Node-{sensor}") == None:
                if algorithm in missing_solutions.keys():
                    missing_solutions[algorithm].append(sensor)
                else:
                    missing_solutions[algorithm] = []
                    missing_solutions[algorithm].append(sensor)

    return missing_solutions    

In [ ]:
print(len(get_missing_solutions()['discrete genetic algorithm']))
print(get_missing_solutions())

In [ ]:
def run_all():
    """
    run_all()
    executes the find_optimal_route() function for all sensors using all algorithms
    for purposes of storing the best solutions for each in the solutions file.
    returns: nothing
    """
    # algorithms to be executed
    methods = ['discrete genetic algorithm', 'ant colony optimization', 'dijkstras algorithm', 'simulated annealing']
    # all sensors except the 2 base stations
    sensors = list(x_y_data.keys())[:-2]
    # for each method, find the best path for each sensor
    for method in methods:
        for sensor in sensors:
            # ignore situations where no best paths will be found. This is typical with DGA and it will kill the function
            try:
                print(f"Sensor {sensor}")
                find_optimal_route(sensor, method)
            except:
                pass        

    return

In [ ]:
#run_all()

### Algorithm Performance Evaluation

The various algorithms that we implemented were evaluated against the following metrics:
- Execution time
- Solution quality

The performances were then visualized in a bar chart and table respectively. The code implementing performance of the algorithms is found in the [Mertics Module](./metrics.ipynb)

#### Execution Time

This metric calculates the time taken in seconds for an algorithm to execute. This measurement when the same type of problem is passed into the various algorithms, for example, getting the best path to any base station from node 43 will be passed in to all the algorithms. The time library was imported to capture execution start time and end time and this the execution time calculated as follows:

`
execution time = execution stop time – execution start time
`

This information is recorded against the algorithm name in a dictionary. After all the executions, the dictionary is used to build a bar chart which shows execution time in seconds as in the figure below.

![Alt](https://github.com/samariwa/search-and-optimization-projects/blob/main/assessment/images/metrics%20chart.png?raw=true)

The function that fetches the execution time performance is shown below. To execute run `get_algorithm_time_metrics()`

In [ ]:
def get_algorithm_time_metrics():
    """
    get_algorithm_time_metrics()
    this function compares the performance of the various algorithms 
    in terms of execution time and plots them on a bar graph. This is done
    by setting a timer at the beginning of execution of an algorithm and stopping
    it at the end of the execution. The duration is stored in a dictionary against
    the algorithm name as a key in the stop_execution() function. These functions
    are found in the found in the metrics module. The durations are plotted on a
    bar graph for visualization purposes.
    returns: nothing
    """
    # choose any random node to start from with the exception of the base stations
    dest_nodes = [x_y_base_station_1[0], x_y_base_station_2[0]]
    start_node = choice(np.array([i for i in x_y_data.keys() if i not in dest_nodes]), 1, replace=False)[0]
    # list of algorithms whose time performance will be compared
    methods = ['discrete genetic algorithm', 'ant colony optimization', 'dijkstras algorithm', 'simulated annealing']
    for method in methods:
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        print(f"executing {method}...")
        # start execution timer
        start_execution()
        # run the algorithm
        find_optimal_route(start_node, method)
        # stop the timer and record the results
        stop_execution(method)
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    # plot the recorded results
    plot_algorithm_execution_time()

    return

In [ ]:
#get_algorithm_time_metrics()

#### Solution Quality

The best paths from a sensor to a base station may vary for various algorithms. These paths may have varying absolute costs or similar absolute costs. This metric analyses the best paths in terms of the absolute cost and ranks them such that those that have a high absolute cost are ranked higher. But this metric being done once may not reflect the consistent generation of quality solutions. To take care of this, we set the algorithm to run several problems and for each problem rank them. At the end, ranks for each of the problems are ranked to come up with an aggregated rank of the best solution which are represented in a table as shown below.

![Alt](https://github.com/samariwa/search-and-optimization-projects/blob/main/assessment/images/metrics%20table.png?raw=true)

In the figure above, Dijkstra and simulated annealing got the best solutions and tied in terms of their performance ranking. The solution quality evaluation is implemented using the function below. To execute run `get_algorithm_accuracy_metrics()` with the number of trials passed in as an argument (Default is 4).

In [ ]:
def get_algorithm_accuracy_metrics(trials=4):
    """
    get_algorithm_accuracy_metrics()
    ranks the algorithms in terms of the quality of the solutions they provide.
    The ranking is done by running the several runs of all algorithms, ranking 
    the individual ranks then ranking the ranks in order to get a wholistic rank
    of the algorithms. The ranks are then displayed on a table. Rank ties 
    could exist in some scenarios
    trials (optional): number of times you want to run the algorithm to get an
    aggregated rank for more consistency in performance ranking
    returns: nothing
    """
    # choose any random node to start from with the exception of the base stations
    dest_nodes = [x_y_base_station_1[0], x_y_base_station_2[0]]
    # list of algorithms whose time performance will be compared
    methods = ['discrete genetic algorithm', 'ant colony optimization', 'dijkstras algorithm', 'simulated annealing']
    accuracy_performance = []
    # initialize the sum of ranks for overall ranking
    rank_sum = np.zeros(len(methods))
    trial_no = trials
    # create a store for the final rank data
    overall_performance = []
    for trial in range(trial_no):
        start_node = choice(np.array([i for i in x_y_data.keys() if i not in dest_nodes]), 1, replace=False)[0]
        print(f"trial {trial + 1}...")
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        path_costs = []
        for method in methods:      
            print(f"executing {method}; trial {trial + 1}...")
            '''
            run the algorithm and get the solutions data with contains 
            the albolute cost which is used to dictate the quality of 
            the solution. The higher the cost the better the rank
            '''
            if method == "discrete genetic algorithm":
                results, solutions = discrete_genetic_algorithm(start_node)
            elif method == "dijkstras algorithm":
                results, solutions = dijkstra_algorithm(start_node)
            elif method == "ant colony optimization":
                results, solutions = ant_colony_optimization(start_node)
            elif method == "simulated annealing":
                results, solutions = simulated_annealing(start_node)
            path_costs.append(float(solutions[0]['absolute path cost']))
        # rank the path costs. negate the costs so that ranking is done in descending order
        ranked_costs = stat.rankdata([-cost for cost in path_costs])
        accuracy_performance.append(list(map(int, ranked_costs)))
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    #compile the individual trial ranks into a single wholistic rank
    for ranks in range(len(accuracy_performance)):
        for method in range(len(methods)):
            rank_sum[method] += accuracy_performance[ranks][method]
    # rank the sum of the individual ranks
    rank_data = list(map(int,stat.rankdata(rank_sum)))
    for i in range(len(rank_data)):
        overall_performance.append([rank_data[i], methods[i]])
    # tabulate the overall rank
    show_algorithm_accuracy_rank(overall_performance)

    return

In [ ]:
#get_algorithm_accuracy_metrics()

#### Evaluation

After running several tests, we had the following results for our algorithms:
- Execution time metric:
    1. Dijkstra’s Algorithm (< 1 second)
    2. Simulated Annealing Algorithm (1-5 seconds)
    3. Discrete Genetic Algorithm (20 – 60seconds)
    4. Ant’s Colony Optimization Algorithm (200-250 seconds)

The indicated times ranges are where most execution times lie for round of algorithm execution. They vary based on the kind of randomly generated solutions in the cases of the stochastic algorithms like Simulated Annealing and Discrete Genetic Algorithm. These times also vary based on the **kwargs provided e.g. the population size in DGA or the number of ants in ACO. These performances were run against the default parameters which had been pre-tested to ensure optimal output.
Based on the results, Dijkstra’s Algorithm is the best algorithm for this problem in terms of time since executes in the shortest time consistently.

- Solution quality metric:
    1. Dijkstra’s Algorithm
    2. Simulated Annealing
    3. Discrete Genetic Algorithm
    4. Ant Colony Optimization

The ranks were done based on 4 trials for each algorithm, and for each trial, the best path from a particular node was tried for each algorithm. There were times when DGA emerged the best because it is dependent on the randomly generated initial paths and the permutation with is more of a trial and error. The above ranks were derived from several executions and the performance consolidated to the above.
Based on the results, Dijkstra’s Algorithm performed best for this problem because it consistently gave good results. The upside of Dijkstra’s algorithm is the logic of evaluating the next best path in the trail. This made the algorithm always get to the base station from a node in a particular route even when run several times. This consistency makes it a reliable algorithm for good solutions alongside the quick execution time as discussed above.

#### Tradeoff between time and quality

These rankings were based on several tests but again they vary based on the **kwargs provided. At the end, for stochastic algorithms, there is a tradeoff between time and quality. For example, if you initialize many ants in the ACO algorithm, more time will be used to explore the search trail and therefore to get the best path and you might get better solutions.

In [ ]:
for _ in range(10):
    for algorithm, sensors in get_missing_solutions().items():
        for sensor in sensors:
            try:
                print(f"Sensor {sensor}")
                find_optimal_route(sensor, algorithm)
            except:
                pass

## Conclusion

## References

Rosenberg, M., 2024. The Distance Between Degrees of Latitude and Longitude. Available from: https://www.thoughtco.com/degree-of-latitude-and-longitude-distance-4070616 [Accessed 6th January 2024] <br>
Scripts, M. T., 2024. Calculate distance, bearing and more between Latitude/Longitude points. Available from: https://www.movable-type.co.uk/scripts/latlong.html [Accessed 6th January 2024] <br>
Story, R., 2013. Folium. Available from: https://python-visualization.github.io/folium/latest/ [Accessed 6th January 2024] <br>
